In [8]:
import requests
import json
import pandas as pd
import os
import sys
import urllib.parse
import nltk
import string
import copy
import re
import numpy as np
from collections import defaultdict

In [50]:
#1. Consider only the alphabetic part of the LCC code
def lcc_to_scimago_area_category(lcc_code):
    alphabetic_code = re.findall('[a-zA-Z]+', lcc_code)
    if len(alphabetic_code) == 0:
        print("Error: invalid LCC code "+lcc_code+": doesn't start with alphabetic characters")

    alphabetic_code = alphabetic_code[0].upper()
    query_df = lcc_lookup_df.loc[lcc_lookup_df['lcc_code'] == alphabetic_code]
    lcc_subject = None
    if len(query_df) > 0:
        lcc_subject = query_df["lcc_subject"].values[0]
    else:
        return ("", "")
        #print("Error: invalid LCC code "+lcc_code+": not found")

    area = ""
    category = ""
    #2. Checks whether the value of the LCC subject is also a Scimago subject area
    query_df = scimago_lookup_df.loc[scimago_lookup_df['area'].str.lower() == lcc_subject.lower()]
    if len(query_df) > 0:
        area = query_df["area"].values[0]
        category = area + " (miscellaneous)"

    #3. Checks whether the value of the LCC subject is also a Scimago subject category
    # some category names might be included in more than one area -> We take into consideration all the alternativies
    else:
        query_df = scimago_lookup_df.loc[scimago_lookup_df['category'].str.lower() == lcc_subject.lower()]
        if len(query_df) > 0:
            area = ""
            category = ""
            for a_index in range(0,len(query_df["area"].values)):
                area = area + query_df["area"].values[a_index] +";;"
                category = category + query_df["category"].values[a_index] +";;"
            area = area[:-2]
            category = category[:-2]

    return (area, category)

In [51]:
lcc_lookup_df = pd.read_csv("script/data/lcc_lookup.csv")
scimago_lookup_df = pd.read_csv("script/data/scimago_lookup.csv")
df_all_cits_inp1 = pd.read_csv('annotation/all_cits(normalized).csv')

l_rows = []
#cited_magid	cited_doi	cited_ref_hints(author)	cited_retraction_year	cited_publication_year	citing_doi	citing_title	citing_author	citing_venue	venue_id	venue_type	venue_area	venue_category	citing_publisher	citing_publication_year	source	period	fulltext	retracted	notes
for k, row in df_all_cits_inp1.iterrows():
    venue_id = str(row["venue_id"]).lower()
    if venue_id.startswith("isbn"):
        if "lcc" in venue_id:
            lcc_code = venue_id.split("lcc:")[1].strip()
            scimago_res = lcc_to_scimago_area_category(lcc_code)
            row["venue_area"] = scimago_res[0]
            row["venue_category"] = scimago_res[1]
    l_rows.append(row)

In [52]:
pd.DataFrame(l_rows).to_csv('annotation/all_cits(lcc_to_scimago).csv',index=False)

In [53]:
scimago_lookup_df

,area,category
0,Agricultural and Biological Sciences,Agronomy and Crop Science
1,Agricultural and Biological Sciences,Animal Science and Zoology
2,Agricultural and Biological Sciences,Aquatic Science
3,Agricultural and Biological Sciences,"Ecology, Evolution, Behavior and Systematics"
4,Agricultural and Biological Sciences,Food Science
...,...,...
279,Social Sciences,Transportation
280,Social Sciences,Urban Studies
281,Veterinary,Equine
282,Veterinary,Food Animals


In [54]:
lcc_lookup_df

,lcc_code,lcc_subject
0,A,General Works
1,AC,Collections. Series. Collected works
2,AE,Encyclopedias
3,AG,Dictionaries and other general reference works
4,AI,Indexes
...,...,...
225,VG,Minor services of navies
226,VK,Navigation. Merchant marine
227,VM,Naval architecture. Shipbuilding. Marine engin...
228,Z,"Bibliography, Library Science"
